In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from matplotlib.colors import ListedColormap

# Mount google colab to drive to access to the dataset (uncomment if you use Google Colab + Drive)
# from google.colab import drive
# drive.mount('/content/drive/')

from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


# Q1


In [2]:
# TODO: Load dataset
w3classif = pd.read_csv("/content/drive/MyDrive/w3classif.csv", header = None) # Specify full path if you use Google Colab + Drive

In [5]:
# TODO: create a function to create 10 different shuffled train and test set pairs from the original dataset
def create_train_test_data(test_size=0.3):
  # For storing data
  trains, tests = [], []

  for i in range(10):
    # TODO: Shuffle the dataset
    shuffled_data = w3classif.sample(frac=1, random_state=i).reset_index(drop=True)

    # TODO: Split the dataset
    train_data, test_data = train_test_split(shuffled_data, test_size=test_size, random_state=i)

    # Store data
    trains.append(train_data)
    tests.append(test_data)

  return trains, tests



# Q2

In [6]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# TODO: create a function to repeat classification with kNN for multiple trials
def repeat_knn(trials=10, test_size=0.3):

  # For storing
  train_losses, test_losses = [], []

  # Create train and test datasets (calling the function you created previously)
  trains, tests = create_train_test_data(test_size)

  for i in range(trials):
    train_data = trains[i]
    test_data = tests[i]

    # TODO: Split train data into features and target
    X_train = train_data[[0, 1]]
    y_train = train_data[2]


    # TODO: Split test data into features and target
    X_test = test_data[[0, 1]]
    y_test = test_data[2]


    # TODO: Initialize the k-NN classifier
    knn = KNeighborsClassifier(n_neighbors=3)


    # TODO: Train the classifier on the training data
    knn.fit(X_train, y_train)

    # TODO: Make predictions on the training and test data
    y_train_pred = knn.predict(X_train)
    y_test_pred = knn.predict(X_test)


    # TODO: Calculate training and test accuracy
    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)


    # TODO: Calculate training and test loss (misclassification rate)
    train_loss = 1 - train_accuracy
    test_loss = 1 - test_accuracy


    # TODO: Store train and test losses
    train_losses.append(train_loss)
    test_losses.append(test_loss)


  return train_losses, test_losses

In [7]:
# Print the average training and test losses for 10 trials using the function implemented above
train_losses, test_losses = repeat_knn(trials=10, test_size=0.3)
print(f'Avg Training Loss (Misclassification Rate): {np.array(train_losses).mean() * 100:.2f}%')
print(f'Avg Test Loss (Misclassification Rate): {np.array(test_losses).mean() * 100:.2f}%')

"""
Old results:
Avg Training Loss (Misclassification Rate): 1.66%
Avg Test Loss (Misclassification Rate): 3.93%
Test/Train loss increased, likely due to model seeing different data each shuffle.
Taking the average across 10 splits is more reliable. May have gotten lucky with the split before.
"""

Avg Training Loss (Misclassification Rate): 2.50%
Avg Test Loss (Misclassification Rate): 4.58%


# Q3

In [8]:
# TODO: Define all possible test set sizes
test_sizes = [0.1, 0.3, 0.5] # 90/10, 70/30, 50/50

for ts in test_sizes:
  # TODO: Repeat Q1 and Q2 and print the average loss for 10 trials for each test set size
  print(f"\nTest size: {int(ts * 100)}% ({int((1 - ts) * 100)}% training)")

  train_losses, test_losses = repeat_knn(trials=10, test_size=ts)

  print(f"Avg Training Loss: {np.mean(train_losses) * 100:.2f}%")
  print(f"Avg Test Loss: {np.mean(test_losses) * 100:.2f}%")

  # Main findings
  # Less training data means worse generalisation

  # 90/10 split
  # train loss: is higher bc more data to learn from
  # test loss: small test set = more variance.

  #70/30 split (maybe a sweet spot)
  # Train/test loss is balanced due to the split

  #50/50 split
  # Train loss: slightly worse, less data to learn from
  # Test loss: slightly higher


Test size: 10% (90% training)
Avg Training Loss: 2.61%
Avg Test Loss: 5.25%

Test size: 30% (70% training)
Avg Training Loss: 2.50%
Avg Test Loss: 4.58%

Test size: 50% (50% training)
Avg Training Loss: 2.80%
Avg Test Loss: 4.75%


# Q4

In [9]:
# TODO: Define all possible test set sizes
test_sizes = [0.1, 0.3, 0.5] # 90/10, 70/30, 50/50

for ts in test_sizes:
  # TODO: Calculate the sample standard deviation of your training and test set error values over the 10 trials from Q2 and Q3.
  print(f"\nTest size: {int(ts * 100)}% ({int((1 - ts) * 100)}% training)")

  # Repeat the 10 trials for this test size
  train_losses, test_losses = repeat_knn(trials=10, test_size=ts)

  # TODO: Calculate the sample standard deviation of training and test losses
  train_std = np.std(train_losses, ddof=1)
  test_std = np.std(test_losses, ddof=1)

  print(f"Training Loss Std Dev: {train_std * 100:.4f}%")
  print(f"Testing Loss Std Dev: {test_std * 100:.4f}%")

  #Observation
  # 90/10 - very stable but test results vary a lot due to small test set
  # 70/30 - both test/train are moderate (seems balanced)
  # 50/50 - training varies more (less data) but test set is large so more stable variation


Test size: 10% (90% training)
Training Loss Std Dev: 0.4182%
Testing Loss Std Dev: 3.9878%

Test size: 30% (70% training)
Training Loss Std Dev: 0.9221%
Testing Loss Std Dev: 2.1960%

Test size: 50% (50% training)
Training Loss Std Dev: 1.1106%
Testing Loss Std Dev: 0.9204%


# Q5

In [10]:
from sklearn.model_selection import cross_val_score

# TODO: Shuffle the original dataset
data_classif = w3classif.sample(frac=1, random_state=42).reset_index(drop=True)


# TODO: Split the dataset into features and target
X = data_classif[[0, 1]]  # first two columns
y = data_classif[2]       # third column

# TODO: Initialize the k-NN classifier
knn = KNeighborsClassifier(n_neighbors=3)

# TODO: Define the number of folds for cross-validation
num_folds = 10

# TODO: Perform cross-validation
cv_scores = cross_val_score(knn, X, y, cv=num_folds, scoring='accuracy')


# TODO: Calculate mean and standard deviation of cross-validation error
cv_errors = 1 - cv_scores  # convert accuracy to misclassification
mean_error = np.mean(cv_errors)
std_error = np.std(cv_errors, ddof=1)


# TODO: Print the errors
print(f"10-Fold Cross-Validation Mean Error: {mean_error * 100:.2f}%")
print(f"10-Fold Cross-Validation Std Dev: {std_error * 100:.2f}%")

10-Fold Cross-Validation Mean Error: 5.00%
10-Fold Cross-Validation Std Dev: 4.86%
